In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osiris_utils as ou

# Interface with Osiris Input Decks

For many tasks, it might be useful to read simulation parameters directly from the input deck, or algorithmically generate new Osiris input decks (e.g.to run large parameter scans). 

The class `InputDeckIO` allows you to easily perform theses tasks.
You need to provide it two inputs:
- `filename`: path to OSIRIS input deck
- `verbose`: if `True` will print auxiliary information when parsing the input deck


## Reading an input deck

In [3]:
deck = ou.InputDeckIO('example_data/thermal.1d', verbose=True)


Parsing input deck : example_data/thermal.1d
Reading simulation
  random_seed = 0
  wall_clock_limit = "23:50:00"
  wall_clock_check = 2000
  wall_clock_checkpoint = .true.
Reading node_conf
  node_number(1:1) = 4
  if_periodic(1:1) = .true.
Reading grid
  nx_p(1:1) = 500
  coordinates = "cartesian"
Reading time_step
  dt = 0.0099
  ndump = 1
Reading restart
  if_restart = .false.
Reading space
  xmin(1:1) = 0.
  xmax(1:1) = 5.0
  if_move(1:1) = .false.
Reading time
  tmin = 0.0d0
  tmax = 2.5
Reading el_mag_fld
Reading emf_bound
  type(1:2,1) = "open","open"
Reading diag_emf
  ndump_fac = 1
  ndump_fac_ene_int = 1
  reports = "e1","e2","e3"
Reading particles
  interpolation = "linear"
  num_species = 1
Reading species
  name = "electrons"
  num_par_max = 2048
  rqm = -1.0
  num_par_x(1:1) = 64
Reading udist
  uth_type = "thermal"
  uth(1:3) = 0.01,0.01,0.01
Reading profile
  density = 1
  profile_type = "uniform"
Reading spe_bound
Reading diag_species
  ndump_fac_ene = 1
  ndump_fac_

Not only does the object load the different module parameters, but also automatically determines other useful information for downstream tasks.

Examples include:
- `dim`: number of dimensions
- `n_species`: number of species
- `species`: dictionary with `Species` objects with relevant information (e.g. charge, rqm, etc.)

In [4]:
print("Simulation Dimensions:", deck.dim)
print("Number of species:", deck.n_species)
print("Species:", deck.species)

Simulation Dimensions: 1
Number of species: 1
Species: {'electrons': Specie(name=electrons, rqm=-1.0, q=-1.0, m=1.0)}


`InputDeckIO` stores the state of the input deck in the parameter `self._sections`.

You should not access this value directly, instead you can use for example its getter function `self.sections` which returns a deepcopy.

The returned value corresponds to a list of pairs [section_name, dictionary] where the section_name matches the Osiris input deck section name and he dictionary contains the list of (key, values) for the parameters of the section.

In [5]:
print(deck.sections[0])
print(deck.sections[1])
print('Number of sections:', len(deck.sections))

['simulation', {'random_seed': '0', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]
['node_conf', {'node_number(1:1)': '4', 'if_periodic(1:1)': '.true.'}]
Number of sections: 18


Another option is to query the object as if it is an iterator, where the key is the section name.

This will return a list of dictionaries, since multiple sections can have the same name (e.g. you might have multiple species).

Once again, a deepcopy is being returned so editing the returned values of the dictionaries will not change the original `InputDeckIO` object.

In [6]:
print(deck['simulation'])
print(deck['node_conf'])

[{'random_seed': '0', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]
[{'node_number(1:1)': '4', 'if_periodic(1:1)': '.true.'}]


Finally you can also ask for a specific parameter directly with `get_param()`

In [7]:
# this one returns a list since there can be multiple sections with the same name
print(deck.get_param(section='simulation', param='random_seed')[0])
# which is equivalent to doing this
print(deck["simulation"][0]['random_seed'])

0
0


## Editing an input deck

To safely edit the value of a parameter in an input deck you can use `set_parameter()`.

**Note**: This re-writes the object values!


In [8]:
# edit a parameter already exists
print('Before', deck['simulation'])
deck.set_param('simulation', 'random_seed', value=42)
print('After', deck['simulation'])

Before [{'random_seed': '0', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]
After [{'random_seed': '42', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]


In [9]:
# add a parameter
print("Before", deck["simulation"])
deck.set_param("simulation", "new_parameter", value='HI!', unexistent_ok=True)
print("After", deck["simulation"])

Before [{'random_seed': '42', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]
After [{'random_seed': '42', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.', 'new_parameter': '"HI!"'}]


And you can also delete the parameter using `delete_param()`.

In [10]:
# delete a parameter
print("Before", deck["simulation"])
deck.delete_param("simulation", "new_parameter")
print("After", deck["simulation"])

Before [{'random_seed': '42', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.', 'new_parameter': '"HI!"'}]
After [{'random_seed': '42', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]


Something slighlty more powerful, is the ability to edit a string in the whole input deck (use with care!)

This can be done for example to automatically change multiple parameter values that are shared / depend on an external quantity.

We can do this using the function `set_tag()`.

**Note**: This only works on the parameter values, not section names / parameter names.

In [11]:
# this a dummy example where we change some values to #tag#
print("Before", deck["simulation"])
deck.set_tag(
    '42', # this has to be a string
    '#tag#',
)
deck.set_tag(
    '23:50:00', # this has to be a string
    '#tag#',
)
print("After 1", deck["simulation"])

# and here we change both values at once
deck.set_tag('#tag#', "BOTH CHANGED")
print("After 2", deck["simulation"])
# There is a reason why wall_clock_limit has an extra "" done worry
# it is because it should be a string, while random_seed is an int!
# InputDeckIO handles these things for you

Before [{'random_seed': '42', 'wall_clock_limit': '"23:50:00"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]
After 1 [{'random_seed': '#tag#', 'wall_clock_limit': '"#tag#"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]
After 2 [{'random_seed': 'BOTH CHANGED', 'wall_clock_limit': '"BOTH CHANGED"', 'wall_clock_check': '2000', 'wall_clock_checkpoint': '.true.'}]


## Writing changes to file

Once you did your changes, you can simply generate a new input deck with `print_to_file()`.

In [12]:
deck.print_to_file("edited-deck.1d")
! cat edited-deck.1d

simulation
{
	random_seed = BOTH CHANGED,
	wall_clock_limit = "BOTH CHANGED",
	wall_clock_check = 2000,
	wall_clock_checkpoint = .true.,
}

node_conf
{
	node_number(1:1) = 4,
	if_periodic(1:1) = .true.,
}

grid
{
	nx_p(1:1) = 500,
	coordinates = "cartesian",
}

time_step
{
	dt = 0.0099,
	ndump = 1,
}

restart
{
	if_restart = .false.,
}

space
{
	xmin(1:1) = 0.,
	xmax(1:1) = 5.0,
	if_move(1:1) = .false.,
}

time
{
	tmin = 0.0d0,
	tmax = 2.5,
}

el_mag_fld
{
}

emf_bound
{
	type(1:2,1) = "open", "open",
}

diag_emf
{
	ndump_fac = 1,
	ndump_fac_ene_int = 1,
	reports = "e1", "e2", "e3",
}

particles
{
	interpolation = "linear",
	num_species = 1,
}

species
{
	name = "electrons",
	num_par_max = 2048,
	rqm = -1.0,
	num_par_x(1:1) = 64,
}

udist
{
	uth_type = "thermal",
	uth(1:3) = 0.01, 0.01, 0.01,
}

profile
{
	density = 1,
	profile_type = "uniform",
}

spe_bound
{
}

diag_species
{
	ndump_fac_ene = 1,
	ndump_fac_temp = 1,
	ndump_fac = 1,
	reports = "charge",
	rep_udist = "vfl1", "T11",
}

